In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('input/data/train_data.csv')
train['totaltime'] = np.load('train_col/totaltime.npy')

In [3]:
train.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,totaltime
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,2.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,2.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,2.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,3.0


In [4]:
for i in [2,4,5,6,8,9]:
    train['class'+str(i)] = [j[i] for j in train['assessmentItemID']]
    

In [5]:
train.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,totaltime,class2,class4,class5,class6,class8,class9
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,2.0,6,0,0,1,0,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2.0,6,0,0,1,0,2
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,2.0,6,0,0,1,0,3
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,2.0,6,0,0,1,0,4
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,3.0,6,0,0,1,0,5


In [8]:
train.drop(labels = ['assessmentItemID', 'testId', 'Timestamp'], axis = 1, inplace= True)

In [10]:
train.head()

,userID,answerCode,KnowledgeTag,totaltime,class2,class4,class5,class6,class8,class9
0,0,1,7224,2.0,6,0,0,1,0,1
1,0,1,7225,2.0,6,0,0,1,0,2
2,0,1,7225,2.0,6,0,0,1,0,3
3,0,1,7225,2.0,6,0,0,1,0,4
4,0,1,7225,3.0,6,0,0,1,0,5


In [13]:
train = train[['userID',  'KnowledgeTag', 'totaltime', 'class2', 'class4',
       'class5', 'class6', 'class8', 'class9','answerCode']]

In [14]:
train.head()

,userID,KnowledgeTag,totaltime,class2,class4,class5,class6,class8,class9,answerCode
0,0,7224,2.0,6,0,0,1,0,1,1
1,0,7225,2.0,6,0,0,1,0,2,1
2,0,7225,2.0,6,0,0,1,0,3,1
3,0,7225,2.0,6,0,0,1,0,4,1
4,0,7225,3.0,6,0,0,1,0,5,1


In [17]:
from catboost import CatBoostClassifier, Pool
import wandb
from wandb.keras import WandbCallback
# initialize data
wandb.init(config={"hyper": "parameter"})


from catboost import Pool, CatBoostRegressor

clf_cat = CatBoostRegressor(iterations=10000,
                            learning_rate=0.01,
                            depth=5, 
                            eval_metric='RMSE',
                            random_seed=23,
                            early_stopping_rounds=200,
                            logging_level='Verbose')

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train[['userID',  'KnowledgeTag', 'totaltime', 'class2', 'class4',
       'class5', 'class6', 'class8', 'class9']], 

                                                    train['answerCode'], 

                                                    test_size=0.2, 

                                                    shuffle=False, 

                                                    random_state=42)

In [42]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1813268, 9), (1813268,), (453318, 9), (453318,))

In [43]:
from catboost import CatBoostRegressor


In [38]:
X_train.shape[1]

9

In [40]:
model = CatBoostRegressor(iterations=500, depth=10, learning_rate=0.1, random_seed=42,
            verbose=50)
model.fit(X_train, y_train, cat_features= list(range(9)),
    eval_set=(X_test, y_test),)

0:	learn: 0.4360077	test: 0.4476553	best: 0.4476553 (0)	total: 317ms	remaining: 31.4s
50:	learn: 0.4125353	test: 0.4738766	best: 0.4407453 (5)	total: 10.1s	remaining: 9.71s
99:	learn: 0.4085540	test: 0.4847353	best: 0.4407453 (5)	total: 19.6s	remaining: 0us

bestTest = 0.4407452868
bestIteration = 5

Shrink model to first 6 iterations.


In [33]:
train.totaltime = train.totaltime.astype('int64')

In [34]:
train.head()

,userID,KnowledgeTag,totaltime,class2,class4,class5,class6,class8,class9,answerCode
0,0,7224,2,6,0,0,1,0,1,1
1,0,7225,2,6,0,0,1,0,2,1
2,0,7225,2,6,0,0,1,0,3,1
3,0,7225,2,6,0,0,1,0,4,1
4,0,7225,3,6,0,0,1,0,5,1


In [42]:
import numpy as np
d = {4:10, 5:20, 6:30, 8:40, 9:50}
for i in d.keys():
    a = np.load('test_col/assessmentItemID_'+str(i)+'.npy', allow_pickle = True)
    a = a.astype('int64')
    a+= d[i]
    np.save('test_col/assessmentItemID_'+str(i)+'.npy', a)


In [24]:
a += 50

In [25]:
a

array([51, 52, 53, ..., 52, 53, 54])

In [27]:
np.save('train_col/assessmentItemID_9.npy', a)

In [43]:
np.load('test_col/assessmentItemID_6.npy')

array([33, 33, 33, ..., 30, 30, 30])

In [31]:
import pandas as pd
df = pd.read_csv('input/data/train_data.csv')


In [32]:
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225


In [33]:
a = [i[8] for i in df['assessmentItemID']]

In [36]:
b = np.array(a).astype('int64')

In [38]:
b += 40

In [49]:
import numpy as np

a = np.load('test_col/item_correct.npy')
b = []
for i in a:
    if i>0.5:
        b.append(1)
    else:
        b.append(0)

In [51]:
np.save('test_col/item_correct.npy', np.array(b))

In [55]:
np.load('test_col/assessmentItemID_8.npy')

array([40, 40, 40, ..., 40, 40, 40])

In [2]:
import numpy as np
a = np.load('train_col/test_correct.npy', allow_pickle=True)
np.unique(a)

array([2, 3])

In [5]:
a = np.load('test_col/test_correct.npy', allow_pickle=True)
b = []
for i in a:
    if i>2.5:
        b.append(3)
    else:
        b.append(2)

In [6]:
np.save('test_col/test_correct.npy',np.array(b))

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
